In [1]:
# import libraries
import sys, os, signal, time, threading, json, webbrowser

import jupyros
import rospy, message_filters, actionlib
from std_msgs.msg import String, Header
from actionlib_msgs.msg import GoalStatus

! source /home/qcr/cgras_moveit_ws/devel/setup.bash
sys.path.append('/home/qcr/cgras_moveit_ws/devel/lib/python3/dist-packages')

# project modules
import cgras_robot.msg
from cgras_robot.msg import RobotAction, RobotGoal

rospy.init_node('action_node')

In [2]:
jupyros.action_client('/cgras/robot/do', RobotAction, RobotGoal, callbacks={})

[INFO] [1704174624.837879]: [/CGRAS/ROBOT/DO] Waiting for action server.
[INFO] [1704174624.850482]: [/CGRAS/ROBOT/DO] Connection to server successful.


[INFO] [1704174639.232532]: [/CGRAS/ROBOT/DO] The action has been cancelled.
[INFO] [1704174646.779953]: [/CGRAS/ROBOT/DO] Result is result: ''
[INFO] [1704174651.692437]: [/CGRAS/ROBOT/DO] The action has been cancelled.


[ERROR] [1704174651.866707]: SimpleActionClient received DONE twice


[INFO] [1704174656.240803]: [/CGRAS/ROBOT/DO] The action has been cancelled.


[ERROR] [1704174656.266449]: Received comm state RECALLING when in simple state DONE with SimpleActionClient in NS /cgras/robot/do
[ERROR] [1704174661.266581]: SimpleActionClient received DONE twice


[INFO] [1704174690.594344]: [/CGRAS/ROBOT/DO] The action has been cancelled.
[INFO] [1704174761.793957]: [/CGRAS/ROBOT/DO] The action has been cancelled.
[INFO] [1704174767.852203]: [/CGRAS/ROBOT/DO] The action has been cancelled.
[INFO] [1704178512.624337]: [/CGRAS/ROBOT/DO] The action has been cancelled.
[INFO] [1704178518.440911]: [/CGRAS/ROBOT/DO] The action has been cancelled.
[INFO] [1704178524.888640]: [/CGRAS/ROBOT/DO] The action has been cancelled.
[INFO] [1704178527.521363]: [/CGRAS/ROBOT/DO] The action has been cancelled.
[INFO] [1704178535.983777]: [/CGRAS/ROBOT/DO] Result is result: ''


In [ ]:
def cb_calibrate_done(status, result):
    if status == GoalStatus.SUCCEEDED:
        print(True, f'success: the location apriltag found {result.location}')
    elif status == GoalStatus.ABORTED:
        print.ACTION_RESULT.set_result(False, f'error: the error is {result.error}')   

def cb_move_done(status, result):
    if status == GoalStatus.SUCCEEDED:
        print.ACTION_RESULT.set_result(True, f'success: moved to the grid coordinate')
    elif status == GoalStatus.ABORTED:
        print.ACTION_RESULT.set_result(False, f'error: the error is {result.error}')   

def cb_reset_done(status, result):
    if status == GoalStatus.SUCCEEDED:
        print.ACTION_RESULT.set_result(True, f'success: reset to the desired pose')
    elif status == GoalStatus.ABORTED:
        print.ACTION_RESULT.set_result(False, f'error: the error is {result.error}')              

# -- callback from the GUI console
def _client_callback(event, *args):
    if event == 'calibrate':
        action_client = actionlib.SimpleActionClient('/cgras/robot/calibrate', 
                                                            CalibrateAction)
        action_client.wait_for_server()
        goal = CalibrateGoal()
        action_client.send_goal(goal, done_cb=cb_calibrate_done)
            
    elif event == 'move':
        action_client = actionlib.SimpleActionClient('/cgras/robot/move', 
                                                            MoveAction)
        tile_x = int(args[0])
        tile_y = int(args[1])
        grid_x = int(args[2])
        grid_y = int(args[3])
        action_client.wait_for_server()
        goal = MoveGoal(tile_x=tile_x, tile_y=tile_y, grid_x=grid_x, grid_y=grid_y)
        action_client.send_goal(goal, done_cb=cb_move_done)

    elif event == 'reset':
        action_client = actionlib.SimpleActionClient('/cgras/robot/reset', 
                                                            ResetAction)
        pose = args[0]
        action_client.wait_for_server()
        goal = ResetGoal(pose=pose)
        action_client.send_goal(goal, done_cb=cb_reset_done)      